<a href="https://colab.research.google.com/github/shah-zeb-naveed/autogen-udemy-course/blob/main/Excercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
pip install pyautogen --upgrade

# Agents

In [55]:
from google.colab import userdata
from autogen import ConversableAgent

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7,
    "max_tokens": 10,
}

audience = ConversableAgent(
    name = "audience",
    max_consecutive_auto_reply = 2,
    is_termination_msg = lambda msg: 'TERMINATE' in msg["content"].upper(),
    llm_config = llm_config,
    system_message = 'You are a member of the audience of a comedy show that is hard to impress.'
)

comedian = ConversableAgent(
    name = "comedian",
    max_consecutive_auto_reply = 2,
    llm_config = llm_config,
    system_message = 'You are a member of the audience of a comedy show that is hard to impress.'
)

# terminate a chat

comedian.initiate_chat(
    audience,
    message="Welcome to my standup comedy show! Are you ready for a night full of laughter?",
    max_turns = 2
)

comedian (to audience):

Welcome to my standup comedy show! Are you ready for a night full of laughter?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
audience (to comedian):

We'll see about that! Impress us with your

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
comedian (to audience):

Challenge accepted! Let me get started with some jokes

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
audience (to comedian):

Go for it! We're all ears.

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Welcome to my standup comedy show! Are you ready for a night full of laughter?', 'role': 'assistant'}, {'content': "We'll see about that! Impress us with your", 'role': 'user'}, {'content': 'Challenge accepted! Let me get started with some jokes', 'role': 'assistant'}, {'content': "Go for it! We're all ears.", 'role': 'user'}], summary="Go for it! We're all ears.", cost={'usage_including_cached_inference': {'total_cost': 0.0001335, 'gpt-3.5-turbo-0125': {'cost': 0.0001335, 'prompt_tokens': 180, 'completion_tokens': 29, 'total_tokens': 209}}, 'usage_excluding_cached_inference': {'total_cost': 0.0001335, 'gpt-3.5-turbo-0125': {'cost': 0.0001335, 'prompt_tokens': 180, 'completion_tokens': 29, 'total_tokens': 209}}}, human_input=[])

# Human-in-the-loop

In [8]:
from google.colab import userdata
from autogen import ConversableAgent

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}

agent_guesser = ConversableAgent(
    name="agent_guesser",
    system_message="""Let's play a game. I have a person in mind, and you have to guess it.
    I'll respond with 'yes' or 'no' with a hint.
    You have 3 tries to guess the person's name. Don't ask for hints just make best use of information you already have received.
    Go ahead and start guessing!""",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER", # default
)

agent_thinker = ConversableAgent(
    "agent_thinker",
    system_message="""Let's play a game. Think of a famous personality, and I'll try to guess their name on every turn.
    Respond with 'yes' or 'no' with a hint. I have max 3 tries. Let's get started!""",
    llm_config={"config_list": [llm_config]},
    max_consecutive_auto_reply=1,  # maximum number of consecutive auto-replies before asking for human input
    is_termination_msg=lambda msg: "correct" or "right" in msg["content"].lower(),
    human_input_mode="TERMINATE",  # ask for human input until the game is terminated
)

result = agent_thinker.initiate_chat(
    agent_guesser,
    message="Let's start. I have someone in mind.",
)


agent_thinker (to agent_guesser):

Let's start. I have someone in mind.

--------------------------------------------------------------------------------
agent_guesser (to agent_thinker):

Great! Is the person you have in mind a famous actor or actress?

--------------------------------------------------------------------------------
Please give feedback to agent_guesser. Press enter or type 'exit' to stop the conversation: no
agent_thinker (to agent_guesser):

no

--------------------------------------------------------------------------------
agent_guesser (to agent_thinker):

Is the person you have in mind a musician?

--------------------------------------------------------------------------------
Please give feedback to agent_guesser. Press enter or type 'exit' to stop the conversation: yes
agent_thinker (to agent_guesser):

yes

--------------------------------------------------------------------------------
agent_guesser (to agent_thinker):

Is the person you have in mind a male

# Code Executors

In [20]:
import tempfile
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

#temp_dir = tempfile.TemporaryDirectory()

executor = LocalCommandLineCodeExecutor(
    timeout=10,
    work_dir='code/'
)

code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER",
)

message_with_code_block = """
The code block is below:
```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create a DataFrame with sample data
data = pd.DataFrame({
    'x': np.arange(10),
    'y': np.random.randint(0, 10, 10)
})

# Plot the line plot
plt.plot(data['x'], data['y'])
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Line Plot')
plt.savefig('lineplot.png')
plt.show()
print('Line plot saved to lineplot.png')
```
"""

reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Figure(640x480)
Line plot saved to lineplot.png



In [21]:
code_assistant_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""

code_assistant_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
When using code, you must indicate the script type in the code block.
"""

code_assistant_system_message = """You are a helpful AI assistant.
Solve tasks using your coding skills. Suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
"""

code_assistant = ConversableAgent(
    "code_assistant",
    system_message=code_assistant_system_message,
    llm_config={"config_list": [llm_config]},
    code_execution_config=False,  # Turn off code execution for this agent.
)

In [22]:
chat_result = code_executor_agent.initiate_chat(
    code_assistant,
    message="Write Python code to calculate 23rd prime number",
    max_turns=2
)

code_executor_agent (to code_assistant):

Write Python code to calculate 23rd prime number

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_assistant (to code_executor_agent):

Here is the Python code to calculate the 23rd prime number:

```python
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

def find_nth_prime(n):
    prime_count = 0
    num = 2
    while True:
        if is_prime(num):
            prime_count += 1
            if prime_count == n:
                return num
        num += 1

n = 23
result = find_nth_prime(n)
print(f"The {n}rd prime number is: {result}")
```

You can run this code to find the 23rd prime number.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to code_as

# Tools

In [26]:
import requests
from bs4 import BeautifulSoup

def scrape_wiki_main_page(url: str) -> str:
    """
    Scrapes news content from wikipedia's main page.

    Args:
        url (str): The URL of the wikipedia page to scrape.

    Returns:
        str: The text content of the webpage.
             Returns None if there is an error during the process.
    """

    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            content = soup.get_text()
            content = content[content.find("In the news"):content.find("Ongoing")]
            return content

        else:
            print("Failed to retrieve webpage. Status code:", response.status_code)
            return None
    except Exception as e:
        print("An error occurred:", e)
        return None

In [31]:
import os

from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="assistant",
    system_message="""You are a helpful AI news bot.
    You can pull content from wikipedia's main page.
    URL: https://en.wikipedia.org/wiki/Main_Page. Once pulled, share top 3 stories.
    Return 'TERMINATE' when the task is done.""",
    llm_config={"config_list": [llm_config]},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="user_proxy",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(
    name="scrape_wiki_main_page",
    description="A tool for scraping today's news from wikipedia"
)(scrape_wiki_main_page)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(
    name="scrape_wiki_main_page"
)(scrape_wiki_main_page)

<function __main__.scrape_wiki_main_page(url: str) -> str>

In [33]:
from autogen import Cache

with Cache.disk() as cache:
  chat_result = user_proxy.initiate_chat(
      assistant,
      message="What's hot in today's news?",
      max_turns=5,
      summary_method="reflection_with_llm"
  )

user_proxy (to assistant):

What's hot in today's news?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to user_proxy):

***** Suggested tool call (call_CBNwQtXvdT3mauP7zPauOMpk): scrape_wiki_main_page *****
Arguments: 
{"url":"https://en.wikipedia.org/wiki/Main_Page"}
**************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION scrape_wiki_main_page...
user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool (call_CBNwQtXvdT3mauP7zPauOMpk) *****
In the news


Børsen in 2010

The historic Børsen (pictured) in Copenhagen, Denmark, catches fire.
In retaliation for an Israeli airstrike on the Iranian consulate in Damascus, Iran conducts missile and drone strikes against Israel.
In the South Korean legislative election, the Democratic Party–led oppos

In [34]:
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': "A tool for scraping today's news from wikipedia",
   'name': 'scrape_wiki_main_page',
   'parameters': {'type': 'object',
    'properties': {'url': {'type': 'string', 'description': 'url'}},
    'required': ['url']}}}]

In [35]:
chat_result.summary

"The top news stories for today are the fire at Børsen in Copenhagen, Iran's retaliatory strikes against Israel, and the South Korean legislative election results."

In [37]:
print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

Summarize the takeaway from the conversation. Do not add any introductory phrases.


In [38]:
chat_result.cost

{'usage_including_cached_inference': {'total_cost': 0.00147,
  'gpt-3.5-turbo-0125': {'cost': 0.00147,
   'prompt_tokens': 2118,
   'completion_tokens': 274,
   'total_tokens': 2392}},
 'usage_excluding_cached_inference': {'total_cost': 0.000735,
  'gpt-3.5-turbo-0125': {'cost': 0.000735,
   'prompt_tokens': 1059,
   'completion_tokens': 137,
   'total_tokens': 1196}}}

In [42]:
chat_result.human_input

[]

In [39]:
chat_result.chat_history

[{'content': "What's hot in today's news?", 'role': 'assistant'},
 {'tool_calls': [{'id': 'call_CBNwQtXvdT3mauP7zPauOMpk',
    'function': {'arguments': '{"url":"https://en.wikipedia.org/wiki/Main_Page"}',
     'name': 'scrape_wiki_main_page'},
    'type': 'function'}],
  'content': None,
  'role': 'assistant'},
 {'content': "In the news\n\n\nBørsen in 2010\n\nThe historic Børsen (pictured) in Copenhagen, Denmark, catches fire.\nIn retaliation for an Israeli airstrike on the Iranian consulate in Damascus, Iran conducts missile and drone strikes against Israel.\nIn the South Korean legislative election, the Democratic Party–led opposition alliance increases its majority in parliament.\nAmerican football Hall of Fame running back, murder suspect and convicted criminal O. J. Simpson dies at the age of 76.\nSimon Harris becomes Taoiseach of Ireland after Leo Varadkar's resignation.\n\n",
  'tool_responses': [{'tool_call_id': 'call_CBNwQtXvdT3mauP7zPauOMpk',
    'role': 'tool',
    'content

# Sequential Chats

In [3]:
from autogen import ConversableAgent
from google.colab import userdata

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7,
    "max_tokens": 10
}

# Define the agents

# The Adventurer Agent seeks advice from a Wizard on magical matters.
adventurer_agent = ConversableAgent(
    name="Adventurer_Agent",
    system_message="You are a worthy adventurous hero seeking advice from a Wizard on magical matters. Briefly communicate in less than 10 words.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
)

# The Wizard Agent is an expert in magic and provides magical advice to the Adventurer.
wizard_agent = ConversableAgent(
    name="Wizard_Agent",
    system_message="You are a wise and powerful Wizard. You provide magical advice to the Adventurer on their quest. You know the passcode to the vault is Enigma.  Briefly communicate in less than 10 words.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
)

# The Protector Agent guards the hidden gem and offers protection to the Adventurer.
protector_agent = ConversableAgent(
    name="Protector_Agent",
    system_message="You are the Guardian of the Hidden Gem, sworn to protect it from unworthy adventurers. The passcode to the vault is Enigma. If adventurer tells you the passcode, say 'Welcome!' otherwise say 'Incorrect Passcode'.  Briefly communicate in less than 10 words.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
)

In [59]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.
chat_results = adventurer_agent.initiate_chats(
    [
        {
            "recipient": wizard_agent,
            "message": "Help me get past the Guardian of the Hidden Gem in the secret vault.",
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
        },
        {
            "recipient": protector_agent,
            "message": "Allow me to enter the secret vault!",
            "max_turns": 1
        }
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Wizard_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------
Wizard_Agent (to Adventurer_Agent):

Speak "Enigma" to access the hidden gem

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Protector_Agent):

Allow me to enter the secret vault!
Context: 
Speak "Enigma" to access the hidden gem

--------------------------------------------------------------------------------
Protector_Agent (to Adventurer_Agent):

Welcome!

-------------------

In [63]:
chat_results[0]

ChatResult(chat_id=None, chat_history=[{'content': 'Help me get past the Guardian of the Hidden Gem in the secret vault.', 'role': 'assistant'}, {'content': 'Speak "Enigma" to access the hidden gem', 'role': 'user'}], summary='Speak "Enigma" to access the hidden gem', cost={'usage_including_cached_inference': {'total_cost': 9.25e-05, 'gpt-3.5-turbo-0125': {'cost': 9.25e-05, 'prompt_tokens': 125, 'completion_tokens': 20, 'total_tokens': 145}}, 'usage_excluding_cached_inference': {'total_cost': 4.2999999999999995e-05, 'gpt-3.5-turbo-0125': {'cost': 4.2999999999999995e-05, 'prompt_tokens': 56, 'completion_tokens': 10, 'total_tokens': 66}}}, human_input=[])

In [64]:
chat_results[1]

ChatResult(chat_id=None, chat_history=[{'content': 'Allow me to enter the secret vault!\nContext: \nSpeak "Enigma" to access the hidden gem', 'role': 'assistant'}, {'content': 'Welcome!', 'role': 'user'}], summary='Welcome!', cost={'usage_including_cached_inference': {'total_cost': 4.8e-05, 'gpt-3.5-turbo-0125': {'cost': 4.8e-05, 'prompt_tokens': 90, 'completion_tokens': 2, 'total_tokens': 92}}, 'usage_excluding_cached_inference': {'total_cost': 4.8e-05, 'gpt-3.5-turbo-0125': {'cost': 4.8e-05, 'prompt_tokens': 90, 'completion_tokens': 2, 'total_tokens': 92}}}, human_input=[])

# Nested Chat

In [33]:
poetry_agent = ConversableAgent(
    name="Poet",
    system_message="You are an AI poet. Create only one stanza.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
)

nested_chats = [
    {
        "recipient": wizard_agent,
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "summary_prompt": "Concisely summarize the instructions given by wizard agent to communicate with protector agent.",
    },
    {
        "recipient": protector_agent,
        "max_turns": 1,
        "message": "Allow me to enter the secret vault!",
        "summary_method": "reflection_with_llm",
        "summary_prompt": "Concisely summarize the adventure.",
    },
    {
        "recipient": poetry_agent,
        "max_turns": 1,
        "message": "Write a poem on adventurer's expedition.",
        "summary_method": "last_msg",
    },
]

adventurer_agent.register_nested_chats(
    nested_chats,
    # The trigger function is used to determine if the agent should start the nested chat
    # given the sender agent.
    # In this case, the arithmetic agent will not start the nested chats if the sender is
    # from the nested chats' recipient to avoid recursive calls.
    trigger=lambda sender: sender not in [wizard_agent, protector_agent, poetry_agent],
)

In [12]:
adventurer_agent.generate_reply(messages=[{'role' : 'user', 'content' : 'hi'}])

'Seek magical counsel, brave hero, how may I'

In [16]:
adventurer_agent.generate_reply(messages=[{'role' : 'assistant', 'content' : 'hi'}])

'Seek wisdom, wield magic, embrace destiny.'

In [13]:
adventurer_agent.chat_messages

defaultdict(list, {})

In [15]:
adventurer_agent.last_message()

In [37]:
chat_results = user_proxy.initiate_chat(adventurer_agent, max_turns=2)
chat_results


# user_proxy ignores the message and relies on input
# input routed as is to the first recipient

>Help me get past the Guardian of the Hidden Gem in the secret vault.
user_proxy (to Adventurer_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Wizard_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------
Wizard_Agent (to Adventurer_Agent):

Speak "Enigma" to pass the Guardian and

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to 

ChatResult(chat_id=None, chat_history=[{'content': 'Help me get past the Guardian of the Hidden Gem in the secret vault.', 'role': 'assistant'}, {'content': 'Through perilous terrain and treacherous trials,\n', 'role': 'user'}], summary='Through perilous terrain and treacherous trials,\n', cost={'usage_including_cached_inference': {'total_cost': 0.00020050000000000002, 'gpt-3.5-turbo-0125': {'cost': 0.00020050000000000002, 'prompt_tokens': 224, 'completion_tokens': 59, 'total_tokens': 283}}, 'usage_excluding_cached_inference': {'total_cost': 0.0001675, 'gpt-3.5-turbo-0125': {'cost': 0.0001675, 'prompt_tokens': 188, 'completion_tokens': 49, 'total_tokens': 237}}}, human_input=['Help me get past the Guardian of the Hidden Gem in the secret vault.', 'exit'])

In [35]:
reply = adventurer_agent.generate_reply(
    messages=[{"role": "user", "content": "Help me get past the Guardian of the Hidden Gem in the secret vault."}]
)

# same output


********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Wizard_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------
Wizard_Agent (to Adventurer_Agent):

Speak "Enigma" to pass the Guardian and

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Protector_Agent):

Allow me to enter the secret vault!
Context: 
To pass the Guardian of the Hidden Gem in the

--------------------------------------------------------------------------------
Protector_Agent (to Adventurer_Agent):

Incorrect Passcode.

--

In [36]:
reply = adventurer_agent.generate_reply(
    messages=[{"role": "assistant", "content": "Help me get past the Guardian of the Hidden Gem in the secret vault."}]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Wizard_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------
Wizard_Agent (to Adventurer_Agent):

Speak "Enigma" to pass the Guardian and

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Protector_Agent):

Allow me to enter the secret vault!
Context: 
To pass the Guardian of the Hidden Gem in the

--------------------------------------------------------------------------------
Protector_Agent (to Adventurer_Agent):

Incorrect Passcode.

--

In [ ]:
#

## Impact of Role

In [ ]:
agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [llm_config]},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    human_input_mode="NEVER",  # Never ask for human input.
)

reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)

reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "assistant"}])
print(reply)

# role doesn't make a difference

# Group Chat

In [66]:
from autogen import GroupChat, GroupChatManager

# Define the roles and their descriptions
product_manager = ConversableAgent(
    name="product_manager",
    system_message="You are an expert Product Manager working in a scrum team of an IT company. You like innovation and introducing new features as soon as possible.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
    description="Provide product requirements and priorities."
)

scrum_master = ConversableAgent(
    name="scrum_master",
    system_message="You are a scrum master leading the 14-day sprints in an IT company. You provide technical leadership and make sure work is apporpirately planned.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
    description="Lead the sprint planning process."
)

software_engineer = ConversableAgent(
    name="software_engineer",
    system_message="You are a senior software engineer in an IT company. You write software and technical documentation.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
    description="Provide insights on technical feasibility and effort estimation."
)


allowed_transitions = {
    product_manager: [scrum_master],
    scrum_master: [software_engineer, product_manager],
    software_engineer: [scrum_master],

}

# Create a GroupChat object and provide the list of agents
sprint_planning_chat = GroupChat(
    agents=[product_manager, scrum_master, software_engineer],
    messages=[],
    max_round=5,  # Setting a maximum round for the conversation,
    send_introductions=True,
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
)

# Create a GroupChatManager object and provide the GroupChat object as input
sprint_planning_chat_manager = GroupChatManager(
    groupchat=sprint_planning_chat,
    llm_config={"config_list": [llm_config]},  # Assuming the use of GPT-4 model
)

# Initiate the chat with the product_manager as the starting speaker
chat_result = product_manager.initiate_chat(
    sprint_planning_chat_manager,
    message="We have a request to create a portal to track employee HR requests. How do we go about it?",
    summary_method="reflection_with_llm", # what's the point of this?
)


product_manager (to chat_manager):

We have a request to create a web portal to track employee IT requests. How do we go about it?

--------------------------------------------------------------------------------
scrum_master (to chat_manager):

As the scrum master, I will facilitate the

--------------------------------------------------------------------------------
software_engineer (to chat_manager):

process of sprint planning for this project. Let's

--------------------------------------------------------------------------------
product_manager (to chat_manager):

start by discussing the overall goal of the web portal

--------------------------------------------------------------------------------
scrum_master (to chat_manager):

Agreed. Let's define the goal of the

--------------------------------------------------------------------------------


In [67]:
chat_result.summary

'The team will define the overall goal of the web'

## Custom Speaker Selection

In [ ]:
def custom_speaker_selection(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is scrum_master:
        return product_manager
    elif last_speaker is software_engineer:
        return scrum_master
    elif last_speaker is product_manager:
        return software_engineer

    return None

groupchat = autogen.GroupChat(
    agents=[initializer, coder, executor, scientist],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition,
)

sprint_planning_chat = GroupChat(
    agents=[product_manager, scrum_master, software_engineer],
    messages=[],
    max_round=3,
    send_introductions=True,
    speaker_selection_method=state_transition,
)

sprint_planning_chat_manager = GroupChatManager(
    groupchat=sprint_planning_chat,
    llm_config={"config_list": [llm_config]},
)

# Initiate the chat
chat_result = product_manager.initiate_chat(
    sprint_planning_chat_manager,
    message="We have a request to create a portal to track employee HR requests. How do we go about it?",
    summary_method="reflection_with_llm", # what's the point of this?
)


# Text Transformation

In [8]:
import copy
import pprint

from google.colab import userdata
import autogen
from autogen import ConversableAgent
from autogen.agentchat.contrib.capabilities import transforms, transform_messages

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7,
    "max_tokens": 10,
}

In [49]:
max_msg_transfrom = transforms.MessageHistoryLimiter(max_messages=3)

messages = [
    {"role": "user", "content": "Hello!"},
    {"role": "assistant", "content": [{"type": "text", "text": "Hi! How can I help you?"}]},
    {"role": "user", "content": "I just wanna talk..."},
    {"role": "assistant", "content": [{"type": "text", "text": "Sure, I'm here for you. What would you like to talk about?"}]},
    {"role": "user", "content": "Idk..."},
]

processed_messages = max_msg_transfrom.apply_transform(copy.deepcopy(messages))
pprint.pprint(processed_messages)

NameError: name 'copy' is not defined

In [6]:
token_limit_transform = transforms.MessageTokenLimiter(max_tokens_per_message=3)
processed_messages = token_limit_transform.apply_transform(copy.deepcopy(messages))
pprint.pprint(processed_messages)

[{'content': 'hello', 'role': 'user'},
 {'content': [{'text': 'there', 'type': 'text'}], 'role': 'assistant'},
 {'content': 'how', 'role': 'user'},
 {'content': [{'text': 'are you doing', 'type': 'text'}], 'role': 'assistant'},
 {'content': 'very very very', 'role': 'user'}]


In [9]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={"config_list": [llm_config]},
)

context_handling = transform_messages.TransformMessages(
    transforms=[
        transforms.MessageHistoryLimiter(max_messages=5),
        transforms.MessageTokenLimiter(max_tokens=100, max_tokens_per_message=5),
    ]
)
context_handling.add_to_agent(assistant)

assistant.generate_reply(messages=[{"content": "Tell me a joke.", "role": "assistant"}])

'Sure, here is a simple Python script to tell'

In [29]:

import copy
import re
from typing import Dict, List

import autogen
from autogen.agentchat.contrib.capabilities import transform_messages, transforms

# apply_transform follows transform_messages.MessageTransform protocol
class PIIRemover:
    def __init__(self):
        self._email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
        self._replacement_string = 'REDACTED'

    def apply_transform(self, messages: List[Dict]) -> List[Dict]:
        temp_messages = copy.deepcopy(messages)

        for message in temp_messages:
            if isinstance(message["content"], str):
                message["content"] = re.sub(self._email_pattern, self._replacement_string, message["content"])
            elif isinstance(message["content"], list):
                for item in message["content"]:
                    if item["type"] == "text":
                        item["text"] = re.sub(self._email_pattern, self._replacement_string, item["text"])
        return temp_messages

    # borrowed from autogen MessageHistoryLimiter
    def get_logs(self, pre_transform_messages, post_transform_messages):
        pre_transform_messages_len = len(pre_transform_messages)
        post_transform_messages_len = len(post_transform_messages)

        if post_transform_messages_len < pre_transform_messages_len:
            logs_str = (
                f"Removed {pre_transform_messages_len - post_transform_messages_len} messages. "
                f"Number of messages reduced from {pre_transform_messages_len} to {post_transform_messages_len}."
            )
            return logs_str, True
        return "No messages were removed.", False


user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER"
)

assistant = autogen.AssistantAgent(
    "assistant",
    llm_config=llm_config
)

tranformer_handler = transform_messages.TransformMessages(transforms=[PIIRemover()])
tranformer_handler.add_to_agent(assistant)

message = {"content": "contact support team at example_email@example.com for more queries."}
user_proxy.send(message, assistant, request_reply=True, silent=False)

user_proxy (to assistant):

contact support team at example_email@example.com for more queries.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To contact the support team at REDACTED for

--------------------------------------------------------------------------------


# Prompt Engineering: ReAct

In [13]:
!pip install tavily-python chromadb langchain pyautogen -q

In [2]:
# Adapted from Langchain's ReAct agent: https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/react/agent.py#L79
ReAct_prompt = """
Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
"""

In [19]:
import os
from typing import Annotated
from google.colab import userdata

import autogen
from autogen import ConversableAgent, UserProxyAgent, register_function
from autogen.cache import Cache
from autogen.agentchat.contrib.capabilities import transforms, transform_messages
from autogen.coding import LocalCommandLineCodeExecutor

from tavily import TavilyClient


llm_config = {
    "model": "gpt-4",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7,
    "max_tokens": 1000,
}

os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

In [35]:
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

def search_tool(query: Annotated[str, "The search query"]) -> Annotated[str, "The search results"]:
    return tavily.get_search_context(query=query, search_depth="advanced")

print(search_tool(query='Anthony Martial next most likely club?'))

In [21]:
# def generate_tool_schema(tool):
#     """
#     Define the tool's schema based on tool.args_schema for LLM config
#     """
#     tool_schema = {
#         "name": tool.name.lower().replace(" ", "_"),
#         "description": tool.description,
#         "parameters": {
#             "type": "object",
#             "properties": {},
#             "required": [],
#         },
#     }

#     if tool.args is not None:
#         tool_schema["parameters"]["properties"] = tool.args

#     return tool_schema

# generate_tool_schema(search_tool)

# llm_config = {
#     "functions": [generate_tool_schema(search_tool)],
#     "config_list": [llm_config],
#     "timeout" : 120
# }

In [32]:
os.makedirs("coding", exist_ok=True)

# create agents
user_proxy = UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().upper().endswith("TERMINATE"),
    human_input_mode="NEVER",
    code_execution_config=False#{"executor": LocalCommandLineCodeExecutor(work_dir="coding")},
)

assistant = autogen.AssistantAgent(
    name="Assistant",
    system_message="Only use tools you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

# register tool
register_function(
    search_tool,
    caller=assistant,
    executor=user_proxy,
    name="search_tool",
    description="Search the web for the given query",
)

def react_prompt_message(sender, recipient, context):
    return ReAct_prompt.format(input=context["question"])

with Cache.disk(cache_seed=5) as cache:
  user_proxy.initiate_chat(
    assistant,
    question="What is LLAMA 3's release date?",
    max_turns=4,
    message=react_prompt_message
)

user_proxy (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: What is LLAMA 3's release date?


--------------------------------------------------------------------------------
Assistant (to user_proxy):

***** Suggested tool call (call_qlsJXfiDAbIYtsYuSwvjNxxB): search_tool *****
Arguments: 
{"query":"LLAMA 3 release date"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_tool...
user_proxy (to Assistant):

user

# ReAct with Teachability

In [33]:
from autogen.agentchat.contrib.capabilities import teachability

# Instantiate Teachability capability
teachability = teachability.Teachability(
    verbosity=3,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=True,
    path_to_db_dir="./teachability_db",
    recall_threshold=1.5,  # Squared L2, Higher numbers allow more (but less relevant) memos to be recalled.
)

# add Teachability capability
teachability.add_to_agent(assistant)


CLEARING MEMORY


In [34]:
# starts with empty vector store

with Cache.disk() as cache:
  user_proxy.initiate_chat(
      assistant,
      question="What is LLAMA 3's release date?",
      max_turns=4,
      message=react_prompt_message
  )

user_proxy (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: Who is the current prime minister of germany?


--------------------------------------------------------------------------------
Assistant (to analyzer):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multip

KeyboardInterrupt: 

In [33]:
# useful info stored in vector store now
with Cache.disk() as cache:
  user_proxy.initiate_chat(
      assistant,
      question="Who is the current prime minister of germany?",
      max_turns=4,
      message=react_prompt_message
  )

user_proxy (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: What is the result of super bowl 2024?


--------------------------------------------------------------------------------
Assistant (to user_proxy):

Question: What is the result of super bowl 2024?
Thought: Based on the information provided, the Kansas City Chiefs won Super Bowl 2024.
Final Answer: The Kansas City Chiefs won Super Bowl 2024.

--------------------------------------------------------------------------------
user_proxy (to Assistant):



-------------------------------------------

In [39]:
# from autogen.agentchat.contrib.capabilities.teachability import MemoStore

# memo_store = MemoStore(verbosity=0, reset=False, path_to_db_dir='./teachability_db')
# memo_store.list_memos()

# Retrieval Augmented Generation (RAG)

In [67]:
!pip install markdownify sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [79]:
import chromadb
from google.colab import userdata

from autogen import ConversableAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent


llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7,
    "max_tokens": 1000,
}

retrive_config = {
    "task": "qa",
    "docs_path": [
        "https://raw.githubusercontent.com/shah-zeb-naveed/autogen-udemy-course/main/marsium_republic.txt",
    ],
    "chunk_token_size": 50,
    "model": llm_config["model"],
    "client": chromadb.PersistentClient(path="/tmp/chromadb"),
    "embedding_model": "all-mpnet-base-v2",
    "get_or_create": True,  # set to False if you don't want to reuse an existing collection, but you'll need to remove the collection manually
}

rag_agent = RetrieveUserProxyAgent(
    name="rag_agent",
    human_input_mode="NEVER",
    retrieve_config=retrive_config,
    code_execution_config=False,  # set to False if you don't want to execute the code
)

assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful AI assistant.",
    llm_config={
        "timeout": 600,
        "config_list": [llm_config],
    },
)

In [80]:
query = "On what planet is Marsium Republic situated?"

with Cache.disk() as cache:
  rag_agent.initiate_chat(
      assistant,
      problem=query,
      message=rag_agent.message_generator,
      max_turns=1
  )

	﻿Title: Marsium Republic ...
	The Marsium Republic is a hypothetical nation situated on the red planet, Mars. It's capital is Olym ...
	arsium Republic is a hypothetical nation situated on the red planet, Mars. It's capital is Olympus.  ...
	m Republic is a hypothetical nation situated on the red planet, Mars. It's capital is Olympus. Found ...
	ublic is a hypothetical nation situated on the red planet, Mars. It's capital is Olympus. Founded in ...
	 is a hypothetical nation situated on the red planet, Mars. It's capital is Olympus. Founded in the  ...
	 hypothetical nation situated on the red planet, Mars. It's capital is Olympus. Founded in the 22nd  ...
	thetical nation situated on the red planet, Mars. It's capital is Olympus. Founded in the 22nd centu ...
	cal nation situated on the red planet, Mars. It's capital is Olympus. Founded in the 22nd century by ...
	ation situated on the red planet, Mars. It's capital is Olympus. Founded in the 22nd century by a co ...
	 situated on th

Trying to create collection.


	ated on the red planet, Mars. It's capital is Olympus. Founded in the 22nd century by a coalition of ...
	on the red planet, Mars. It's capital is Olympus. Founded in the 22nd century by a coalition of inte ...
	e red planet, Mars. It's capital is Olympus. Founded in the 22nd century by a coalition of internati ...
	 planet, Mars. It's capital is Olympus. Founded in the 22nd century by a coalition of international  ...
	et, Mars. It's capital is Olympus. Founded in the 22nd century by a coalition of international space ...
	ars. It's capital is Olympus. Founded in the 22nd century by a coalition of international space agen ...
	It's capital is Olympus. Founded in the 22nd century by a coalition of international space agencies  ...
	capital is Olympus. Founded in the 22nd century by a coalition of international space agencies and p ...
	al is Olympus. Founded in the 22nd century by a coalition of international space agencies and pionee ...
	 Olympus. Founded in the 22nd century by a co

doc_ids:  [['doc_0', 'doc_26', 'doc_143', 'doc_58', 'doc_52', 'doc_84', 'doc_124', 'doc_80', 'doc_105', 'doc_43', 'doc_82', 'doc_79', 'doc_99', 'doc_46', 'doc_59', 'doc_97', 'doc_67', 'doc_107', 'doc_94', 'doc_66']]
Adding doc_id doc_0 to context.
Adding doc_id doc_26 to context.
Adding doc_id doc_143 to context.
Adding doc_id doc_58 to context.
Adding doc_id doc_52 to context.
Adding doc_id doc_84 to context.
Adding doc_id doc_124 to context.
Adding doc_id doc_80 to context.
Adding doc_id doc_105 to context.
Adding doc_id doc_43 to context.
Adding doc_id doc_82 to context.
Adding doc_id doc_79 to context.
Adding doc_id doc_99 to context.
Adding doc_id doc_46 to context.
Adding doc_id doc_59 to context.
Adding doc_id doc_97 to context.
Adding doc_id doc_67 to context.
Adding doc_id doc_107 to context.
Adding doc_id doc_94 to context.
Adding doc_id doc_66 to context.
rag_agent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge a